Installations

In [ ]:
%pip install matplotlib pandas openpyxl

Imports

In [ ]:
import subprocess
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd

Preparing folder

In [ ]:
examples_folder_path = '../examplesGenerator/'
results_folder_path = f"{examples_folder_path}results/"
subprocess.run(f"mkdir {results_folder_path}", shell=True, stderr=subprocess.DEVNULL)
subprocess.run(f"rm {results_folder_path}*", shell=True, stderr=subprocess.DEVNULL)
subprocess.run(f"cd {examples_folder_path} && python3 examplesGenerator.py 3 6", shell=True, check=True)

Gathering data

In [ ]:
def get_program_compilation_time(file,opt):
    print(f"Compiling {file} with {opt}")
    time_result : subprocess.CompletedProcess = subprocess.run(f"time clang++ -o output.o {opt} {file}",
                                                               check=True, shell=True, capture_output=True, text=True)
    if time_result.returncode != 0:
        print(f"Error in {file}")
        sys.exit(1)
    elapsed_time_str = time_result.stderr.split(' ')[2]
    time_str = elapsed_time_str.replace('elapsed', '')
    time_str_list = time_str.split(':')
    secs = float(time_str_list[-1])
    mins = 0
    if len(time_str_list) >= 2:
        mins = int(time_str_list[-2])
    hours = 0
    if len(time_str_list) >= 3:
        hours = int(time_str_list[0])
    return hours*3600 + mins*60 + secs

def get_program_running_time(file,opt):
    print(f"Running {file} with {opt}")
    time_result : subprocess.CompletedProcess = subprocess.run(f"time ./output.o 0",
                                                               check=True, shell=True, capture_output=True, text=True)
    if time_result.returncode != 0:
        print(f"Error in {file}")
        sys.exit(1)
    elapsed_time_str = time_result.stderr.split(' ')[2]
    time_str = elapsed_time_str.replace('elapsed', '')
    time_str_list = time_str.split(':')
    secs = float(time_str_list[-1])
    mins = 0
    if len(time_str_list) >= 2:
        mins = int(time_str_list[-2])
    hours = 0
    if len(time_str_list) >= 3:
        hours = int(time_str_list[0])
    return hours*3600 + mins*60 + secs

def get_program_size(file) -> int:
    size_result : subprocess.CompletedProcess = subprocess.run(f"size output.o", shell=True, capture_output=True, text=True)
    if size_result.returncode != 0:
        print(f"Error in {file}")
        sys.exit(1)
    size_str = size_result.stdout.split('\n')[1].split('\t')
    return int(size_str[0])

def get_program_cpu_cycles() -> int:
    subprocess.run(f"perf stat -o temp.txt -e cycles ./output.o 0", shell=True, check=True)
    with open('temp.txt', 'r') as f:
        lines = f.readlines()
        lines = [line for line in lines if line.strip() != '']
        words = lines[2].strip().split(' ')
        cycles = int(words[0].replace('.', ''))
    return cycles

In [ ]:
recursive_calls = []
optimizations = []
sizes = []
cpu_cycles = []
running_times = []
compilation_times = []
opts = ['-O0', '-O1', '-O2', '-O3', '-Os', '-Oz', '-Ofast']
for file in os.listdir(results_folder_path):
    info_array = file.split('_')
    production_rule = int(info_array[2])
    for opt in opts:
        recursive_calls.append(production_rule)
        optimizations.append(opt)
        compilation_times.append(get_program_compilation_time(results_folder_path+file,opt))
        running_times.append(get_program_running_time(results_folder_path+file,opt))
        sizes.append(get_program_size(file))
        cpu_cycles.append(get_program_cpu_cycles())
subprocess.run(f"rm output.o temp.txt", shell=True)

Building tables

In [ ]:

df = pd.DataFrame({'Recursive Calls': recursive_calls, 'Optimizations': optimizations, 'Size': sizes, 'CPU Cycles': cpu_cycles, 'Compilation Time': compilation_times, 'Running Time': running_times})
df.sort_values(by=['Recursive Calls', 'Optimizations'], inplace=True)

sizes_table = df.pivot(index='Optimizations', columns='Recursive Calls', values='Size')
sizes_table.reset_index(inplace=True)
sizes_table.columns.name = None

cpu_cycles_table = df.pivot(index='Optimizations', columns='Recursive Calls', values='CPU Cycles')
cpu_cycles_table.reset_index(inplace=True)
cpu_cycles_table.columns.name = None

compilation_time_table = df.pivot(index='Optimizations', columns='Recursive Calls', values='Compilation Time')
compilation_time_table.reset_index(inplace=True)
compilation_time_table.columns.name = None

running_time_table = df.pivot(index='Optimizations', columns='Recursive Calls', values='Running Time')
running_time_table.reset_index(inplace=True)
running_time_table.columns.name = None

subprocess.run(f"mkdir results", shell=True, stderr=subprocess.DEVNULL)
df.to_csv('results/main_table.csv', index=False)
sizes_table.to_csv('results/sizes_table.csv', index=False)
cpu_cycles_table.to_csv('results/cpu_cycles_table.csv', index=False)
compilation_time_table.to_csv('results/compilation_time_table.csv', index=False)
running_time_table.to_csv('results/running_time_table.csv', index=False)

with pd.ExcelWriter('results/tables.xlsx') as writer:
    df.to_excel(writer, sheet_name='Main Table', index=False)
    sizes_table.to_excel(writer, sheet_name='Sizes Table', index=False)
    cpu_cycles_table.to_excel(writer, sheet_name='CPU Cycles Table', index=False)
    compilation_time_table.to_excel(writer, sheet_name='Compilation Time Table', index=False)
    running_time_table.to_excel(writer, sheet_name='Running Time Table', index=False)